In [96]:
import pandas as pd
import numpy as np
import json

import requests
from bs4 import BeautifulSoup

import time
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [91]:

sections = [
    'news/bulletin/',
    'news/international/',
    'news/local-news/',
    'news/newswire/',
    'features/geared-up/',
    'features/outings/',
    'features/tips/'
]

In [92]:
cookies = {
    'PHPSESSID': '2k6ra51aiq8bkoffj41k3i7nl6',
    '_gid': 'GA1.2.60285680.1694179075',
    '__gads': 'ID=62a9074952e63892:T=1691302727:RT=1694179077:S=ALNI_MbifilTODQ4waB-NotcWZmrmqJGCQ',
    '__gpi': 'UID=00000c2790829244:T=1691302727:RT=1694179077:S=ALNI_Mb4tZnjW_oAqV9zASTyNdddCPFd9A',
    '_ga': 'GA1.1.984074356.1691302726',
    '_ga_0MVNT427LM': 'GS1.1.1694179073.2.1.1694179269.0.0.0',
}

headers = {
    'authority': 'www.bikesrepublic.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': 'PHPSESSID=2k6ra51aiq8bkoffj41k3i7nl6; _gid=GA1.2.60285680.1694179075; __gads=ID=62a9074952e63892:T=1691302727:RT=1694179077:S=ALNI_MbifilTODQ4waB-NotcWZmrmqJGCQ; __gpi=UID=00000c2790829244:T=1691302727:RT=1694179077:S=ALNI_Mb4tZnjW_oAqV9zASTyNdddCPFd9A; _ga=GA1.1.984074356.1691302726; _ga_0MVNT427LM=GS1.1.1694179073.2.1.1694179269.0.0.0',
    'referer': 'https://www.bikesrepublic.com/category/features/tips/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

### 1.0 Get number of pages

In [93]:
pages = []
for section in sections:
    print(f'Scraping "{section}"')
    i = 0
    while section is not None:
        i += 1
        url = f"https://www.bikesrepublic.com/category/news/{section}page/{i}"
        response = requests.get(url, cookies=cookies, headers=headers)
        if response.status_code == 200:
            pages.append(url)
        else:
            print(f"'{section.title()}' only has {i} pages.")
            break

pages_unique = list(set(pages))
print(f'Num. of unique pages: {len(pages_unique)}')
with open(f'bikesrepublic.json', 'a') as f:
    json.dump(pages_unique, f)

Scraping "news/bulletin/"


KeyboardInterrupt: 

### 2.0 Get links in each page

In [ ]:
page_nums = []
with open(f'bikesrepublic.json') as fopen:
    links = json.load(fopen)
page_nums.extend(links)

In [84]:
def crawl(url):
    while True:
        try:
            response = requests.get(url, cookies=cookies, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(response.text, "lxml")

    block = soup.find('div',attrs = {"class":"td-ss-main-content"})
    
    if block is None:
        return

    for link in block.find_all('h3', attrs = {"class":"entry-title td-module-title"}):
        try:
            href = link.find('a').get('href')
            hrefs.append(href)
        except:
            pass

In [ ]:
max_worker = 10

hrefs = []
for i in tqdm(range(len(page_nums))):
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, t): t for t in page_nums}

    for future in as_completed(futures):
        future.result()

hrefs2 = list(set(hrefs))
print(f'Num. of unique links: {len(hrefs2)}')
with open(f'bikesrepublic-links.json', 'a') as f:
    json.dump(hrefs2, f)

### 3.0 Get content from articles